In [1]:
import os

In [2]:
import pyiron_base
pyiron_base.__file__

/Users/jan/mambaforge/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/Users/jan/projects/pyiron_base/pyiron_base/__init__.py'

In [3]:
from pyiron_base import Project
from pyiron_base.project.delayed import draw

In [4]:
pr = Project("test")

In [5]:
pr.remove_jobs(recursive=True, silently=True)

100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  9.33it/s]


In [6]:
def write_input(working_directory, input_dict):
    for k, v in input_dict.items():
        with open(os.path.join(working_directory, k + ".txt"), "w") as f:
            f.writelines(str(v))

In [7]:
def collect_output(working_directory):
    with open(os.path.join(working_directory, "result.txt"), "r") as f:
        return {"result": int(f.readlines()[0])}

In [8]:
pr.remove_jobs(recursive=True, silently=True)

0it [00:00, ?it/s]


In [9]:
z = pr.wrap_executable(
    job_name="job_xy",
    executable_str="x=$(cat x.txt); y=$(cat y.txt); echo $(($x + $y)) > result.txt",
    write_input_funct=write_input,
    collect_output_funct=collect_output,
    input_dict={"x": 1, "y": 2},
    conda_environment_path=None,
    conda_environment_name=None,
    input_file_lst=None,
    delayed=True,
    output_file_lst=["result.txt"],
    output_key_lst=["result"],
)

In [10]:
w = pr.wrap_executable(
    job_name="job_xyz",
    executable_str="x=$(cat x.txt); y=$(cat y.txt); z=$(cat result.txt); echo $(($x + $y + $z)) > result.txt",
    write_input_funct=write_input,
    collect_output_funct=collect_output,
    input_dict={"x": 1, "y": z.output.result},
    conda_environment_path=None,
    conda_environment_name=None,
    input_file_lst=[z.files.result_txt],
    delayed=True,
    output_file_lst=["result.txt"],
    output_key_lst=["result"],
)

In [11]:
w.output.result.result()

initialized
The job job_xy was saved and received the ID: 67
initialized
The job job_xyz was saved and received the ID: 68


7